In [1]:
class DisjointSet:
    def __init__(self, vnum):
        self.parent=[-1 for _ in range(vnum)]

    def simple_find(self, i):
        while self.parent[i] >= 0:
            i=self.parent[i]
        return i

    def simple_union(self, i, j):
        self.parent[i]=j

    def collapsing_find(self, i):
        root=trail=lead=None
        # 루트를 찾는다.
        root=i
        while self.parent[root] >= 0:
            root=self.parent[root]

        # 모든 노드를 루트의 자식으로 만든다
        trail=i
        while trail != root:
            lead=self.parent[trail]
            self.parent[trail]=root
            trail=lead

        return root

    def weighted_union(self, i, j):
        """
        paremeters i, j must be roots!
        if size[i] < size[j] then parent[i]=j
        """
        #abs(parent[i])=size[i]
        #temp_cnt는 음수이고 size[i] + size[j]
        temp_cnt=self.parent[i]+self.parent[j]

        #size[i] < size[j]
        if self.parent[i] > self.parent[j]:
            self.parent[i]=j
            self.parent[j]=temp_cnt
        #size[i] > size[j]
        else:
            self.parent[j]=i
            self.parent[i]=temp_cnt

class Edge:
    def __init__(self, u, v, w):
        self.u=u
        self.v=v
        self.w=w

class Graph:
    def __init__(self, vertex_num):
        self.adj_list=[[] for _ in range(vertex_num)]
        self.edge_list=[]

        self.vertex_num=vertex_num

    def add_edge(self, u, v, weight):
        self.adj_list[u].append(v)
        self.adj_list[v].append(u)

        self.edge_list.append(Edge(u, v, weight))

    def MST_kruskal(self):
        mst=Graph(self.vertex_num)        
        ds=DisjointSet(self.vertex_num)
        self.edge_list.sort(key=lambda e: e.w)
        mst_edge_num=0
        edge_idx=0

        while mst_edge_num < self.vertex_num-1:
            edge=self.edge_list[edge_idx]
            if ds.collapsing_find(edge.u) != ds.collapsing_find(edge.v):
                mst.add_edge(edge.u, edge.v, edge.w)
                ds.weighted_union(ds.collapsing_find(edge.u), ds.collapsing_find(edge.v))
                mst_edge_num+=1
            edge_idx+=1

        return mst

    def print_edges(self):
        for edge in self.edge_list:
            print("({}, {}) : {}".format(edge.u, edge.v, edge.w))